In [ ]:
from IPython.core.display import HTML
HTML("<style>.container { width:100% !important; }</style>")

In [ ]:
import pandas as pd
import numpy as np
import json

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import plotly
import plotly.plotly as py
from plotly.graph_objs import *

In [ ]:
plotly.tools.set_credentials_file(username='ferencberes', api_key='AD0liH3BP3AX8abpysyq')

In [ ]:
plotly.offline.init_notebook_mode()

# TODO

show graph with bokeh interactive visualization:
https://bokeh.pydata.org/en/latest/docs/user_guide/graph.html

# What can be expected?

   * links between players of the same match
   * links between twitter accounts and names (@rafaelnadal - rafa, nadal, vamosrafa etc.) - **Good for account identification**
   * links beween players and adjectives (winner, loser, beats etc.)
   * link between accounts of the same tennis player if they are really used

In [ ]:
with open("/mnt/idms/fberes/network/online_ranker/roland_garros_updated_schedule/filtered_true_matches_screen_names.json") as f:
    filtered_screen_names = json.load(f)

In [ ]:
player_accounts = ["@" + n.lower()  for n in filtered_screen_names]

In [ ]:
player_accounts[:10]

occurences_pd = pd.read_csv("/mnt/idms/fberes/network/roland_garros/combined_occ_parsed.csv",sep="|",header=None)

In [ ]:
#occurences_pd = pd.read_csv("/mnt/idms/fberes/network/roland_garros/combined_occ_parsed_24_hours.csv",sep="|",header=None)
occurences_pd = pd.read_csv("/mnt/idms/fberes/network/roland_garros/combined_occ_parsed_6_hours.csv",sep="|",header=None)

In [ ]:
occurences_pd.head()

In [ ]:
word_cols = list(range(4,201,2))
count_cols = list(range(5,202,2))
word_cols[-1]

# TODO: How can be more nodes than rows in the matrix?

In [ ]:
import networkx as nx

## We should model co-occurences as directed graph! only top x frequently mentioned word should matter for a given word!!!

In [ ]:
def occurences_2_graph(occ_df):
    G = nx.DiGraph()
    notnull_indices = ~occ_df.isnull()
    for col_idx in word_cols:
        partial_df = occ_df.ix[notnull_indices[col_idx]]
        edges_with_data = list(zip(partial_df[2],partial_df[col_idx],partial_df[col_idx+1]))
        G.add_weighted_edges_from(edges_with_data,weight='frequency')
    return G

def filter_by_distance(G, initial_set, max_dist):
    intersection_with_nodes = initial_set.intersection(set(G.nodes()))
    print(len(intersection_with_nodes))
    #print(intersection_with_nodes)
    reachable_nodes = intersection_with_nodes
    current_nodes = intersection_with_nodes
    for i in range(max_dist):
        to_be_examined =set([])
        for s in current_nodes:
            neighbors = nx.neighbors(G, s)
            to_be_examined = to_be_examined.union(set(neighbors))
            print(len(to_be_examined))
        current_nodes = to_be_examined.difference(reachable_nodes)
        reachable_nodes = reachable_nodes.union(to_be_examined)
    return nx.subgraph(G, reachable_nodes)      

def filter_by_degree(G, min_degree):
    degs = G.degree()
    filtered_nodes = []
    for n in G.nodes():
        if degs[n] >= min_degree:
            filtered_nodes.append(n)
    return nx.subgraph(G, filtered_nodes)

def filter_graph(G, max_dist_from_players=None, min_degree=None):
    if max_dist_from_players != None:
        G = filter_by_distance(G, set(player_accounts), max_dist_from_players)             
    if min_degree != None:
        G = filter_by_degree(G, min_degree)
    print(G.number_of_edges(), G.number_of_nodes())
    return G

def get_snapshot_graph(snapshot_start_index, max_dist_from_players=0, min_degree=None):
    spec_day = occurences_pd[occurences_pd[0] == snapshot_start_index]
    # NOTE: if the line below is commented out than we would get a huge graph!!!
    spec_day_for_players = spec_day[spec_day[2].isin(player_accounts)]
    #print(len(occurences_pd), len(spec_day_for_players))
    #G = occurences_2_graph(spec_day)#_for_players)
    G = occurences_2_graph(spec_day_for_players)
    #G_filtered = G
    #print(G.number_of_edges(), G.number_of_nodes())
    G_filtered = filter_graph(G, max_dist_from_players=max_dist_from_players, min_degree=min_degree)
    partial_occurences = spec_day[spec_day[2].isin(G_filtered.nodes())]
    return nx.Graph(G_filtered), partial_occurences 

In [ ]:
#snapshot_id = "2017-06-09T00:00"
snapshot_id = "2017-06-11T00:00"

In [ ]:
S, occ = get_snapshot_graph(snapshot_id, max_dist_from_players=1)

In [ ]:
from collections import Counter

In [ ]:
cnt = Counter(S.degree())

In [ ]:
cnt.most_common()

SP = get_snapshot_graph(snapshot_id, max_dist_from_players=2)

In [ ]:
def get_top_k_occ(tmp_df,name,k):
    tmp_2_df = tmp_df[tmp_df[2] == name]
    word_pairs = []
    for idx in tmp_2_df.index:
        word_pairs += list(zip(list(tmp_2_df.ix[idx][word_cols]),list(tmp_2_df.ix[idx][count_cols])))
    tmp_3_df = pd.DataFrame(word_pairs, columns=["word","count"])
    tmp_3_df = tmp_3_df[~tmp_3_df.isnull()["word"]]
    tmp_3_df = tmp_3_df.sort_values("count", ascending=False).reset_index()
    del tmp_3_df["index"]
    out = ""
    for idx, row in tmp_3_df.head(k).iterrows():
        out += "<br> %s: %i" % (row["word"],row["count"])
    return out

In [ ]:
def draw_with_plotly(SP, SP_layout, occ_df, top_k=10, snapshot_id=None):
    edge_trace = Scatter(
    x=[],
    y=[],
    line=Line(width=0.5,color='#888'),
    hoverinfo='none',
    mode='lines')

    for edge in SP.edges():
        x0, y0 = list(SP_layout[edge[0]])
        x1, y1 = list(SP_layout[edge[1]])
        edge_trace['x'] += [x0, x1, None]
        edge_trace['y'] += [y0, y1, None]
        
    node_trace = Scatter(
        x=[],
        y=[],
        text=[],
        mode='markers',
        hoverinfo='text',
        marker=Marker(
            showscale=True,
            colorscale='YIGnBu',
            reversescale=True,
            color=[],
            size=10,
            colorbar=dict(
                thickness=15,
                title='log(Node Connections)',
                xanchor='left',
                titleside='right'
            ),
            line=dict(width=2)))

    for node in SP.nodes():
        x, y = list(SP_layout[node])
        node_trace['x'].append(x)
        node_trace['y'].append(y)

    for node_idx, adjacencies in enumerate(SP.adjacency_list()):
        node_trace['marker']['color'].append(np.log(len(adjacencies)+1))
        node_id = SP.nodes()[node_idx]
        node_info = "%s" % node_id
        node_info += ":%s" % get_top_k_occ(occ_df,node_id,top_k)
        node_trace['text'].append(node_info)
    
    fig = Figure(data=Data([edge_trace, node_trace]),
                 layout=Layout(
                    width=1500,
                    height=800,
                    title='<br>RG17 combined occurences: %s' % snapshot_id if snapshot_id != None else "",
                    titlefont=dict(size=16),
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20,l=5,r=5,t=40),
                    xaxis=XAxis(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=YAxis(showgrid=False, zeroline=False, showticklabels=False)))
    if snapshot_id != None:
        plotly.offline.iplot(fig, filename='combined_occurences_%s' % snapshot_id)
    else:
        plotly.offline.iplot(fig, filename='combined_occurences')

In [ ]:
def get_node_color(word):
    if word in player_accounts:
        color = "y"
    elif "@" in word:
        color = "m"
    else:
        color = "r"
    return color

def get_edge_color(edge):
    src, trg = edge
    if "@" in src and "@" in trg:
        color = "b"
    else:
        color = "k"
    return color

def show_player_graph(snapshot_id, max_dist=1, min_deg=None):
    SP, occ_df = get_snapshot_graph(snapshot_id, max_dist_from_players=max_dist, min_degree=min_deg)
    spring_layout = nx.spring_layout(SP,iterations=200)
    #freq_indegree = SP.degree(weight="frequency")
    #node_map = dict([(n,get_node_color(n)) for n in SP.nodes()])
    #node_ids = list(node_map.keys())
    #node_colors = list(node_map.values())
    #node_sizes = [100*(np.log(freq_indegree[n]+1)**2+1) for n in node_ids]
    #edge_sizes = np.log([e[2] for e in SP.edges(data="frequency")])+1
    #edge_colors = [get_edge_color(e) for e in SP.edges()]
    #plt.figure(figsize=(30,20))
    draw_with_plotly(SP, spring_layout, occ_df=occ_df, top_k=10, snapshot_id=snapshot_id)
    #nx.draw_networkx(SP, spring_layout, nodelist=node_ids, node_size=node_sizes, node_color=node_colors, width=edge_sizes, edge_color=edge_colors, weight='frequency')
    #plt.show()

# June 28 - starting day of the tournaments

show_player_graph("2017-05-28T00:00")#, remove_1_degree_nodes=True)

# June 05 - many players are still in the contest

show_player_graph("2017-06-05T00:00", remove_1_degree_nodes=True)

# June 07 - many Quarterfinals

show_player_graph("2017-06-07T00:00", remove_1_degree_nodes=True)

# June 09 : many semifinals on this day

show_player_graph("2017-06-09T00:00", remove_1_degree_nodes=True)

# June 11 : man final + double finals on this day

In [ ]:
tmp_df = show_player_graph("2017-06-11T00:00")

In [ ]:
tmp_df = show_player_graph("2017-06-11T06:00")

In [ ]:
tmp_df = show_player_graph("2017-06-11T12:00")

In [ ]:
tmp_df = show_player_graph("2017-06-11T18:00")